# README
## Options:
### Pretrained model:

### Tokenization: How to manage OOV?


In [11]:
from nltk.tokenize import TweetTokenizer

import copy
import pandas as pd
import numpy as np
import nltk
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

import gensim
import gensim.models.word2vec as w2v
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [2]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

In [3]:
raw_data = pd.read_csv("./data/_PHEME_text.csv")

In [4]:
def vectorize(line):
    words = []
    for word in line:  # line - iterable, for example list of tokens
        try:
            w2v_idx = w2v_indices[word]
        except KeyError:  # if you does not have a vector for this word in your w2v model, continue
            words.append(list(np.zeros(200,)))
            continue
        words.append(list(w2v_vectors[w2v_idx]))
        if not word:
            words.append(None)

        if len(line) > len(words):
            continue
    return np.asarray(words)


In [12]:
# Tokenization
tweet_tokenizer = TweetTokenizer()
tweet_tokens = []
stop_words = set(stopwords.words('english'))

for sent in raw_data.text:
    sent = re.sub(r"http\S+", "&", sent)
    sent = re.sub(r"@\S+", "@", sent)
    sent = re.sub(r"#\S+", "#", sent)
    sent = re.sub(r'([^\s\w@#&]|_)+', '', sent)
    sent = re.sub('', '', sent.lower())
    # sent = [tweet_tokenizer.tokenize(sent)]
    sent = [tweet_tokenizer.tokenize(sent.lower())]
    temp = [token for token in sent[0] if not token in stop_words]
    tweet_tokens.append([temp])
df_tokens = pd.DataFrame(tweet_tokens, columns=['token'])


In [22]:
df_tokens.head()

,token
0,"[breaking, armed, man, takes, hostage, kosher, grocery, east, paris, &]"
1,"[#, killers, dead, confirmed, gendarmerie]"
2,"[top, french, cartoonists, charb, cabu, wolinski, tignous, confirmed, among, dead, #, #, attack,..."
3,"[police, surrounded, area, #, attack, suspects, believed, &, &]"
4,"[photo, armed, gunmen, face, police, officers, near, #, hq, paris, &, &]"


# DOC2VEC

In [14]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [20]:
df_tokens.isna().value_counts()

token
False    5802
dtype: int64

In [24]:
df_tokens.shape

(5802, 1)

In [25]:
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec
# corpus = api.load('text8')
# wv = api.load('word2vec-google-news-300')
# fasttext-wiki-news-subwords-300'
#  'glove-twitter-200',
model = api.load('glove-twitter-200')

In [ ]:
# # https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit
# # Load Google Pre trained word2vec model
# pretrained_google_news_model = KeyedVectors.load_word2vec_format('input_data/GoogleNews-vectors-negative300.bin', binary=True)

# # Access vectors for specific words with a keyed lookup:
# vector = pretrained_google_news_model['food']
# # see the shape of the vector (300,)
# vector.shape

In [ ]:
w2v_object = model.wv
w2v_vectors = w2v_object.vectors # here you load vectors for each word in your model
w2v_indices = {word: w2v_object.vocab[word].index for word in w2v_object.vocab} # here you load indices - with whom you can find an index of the particular word in your model 

In [ ]:
tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(tokenized_doc)]
tagged_data
